# Pydantic fundamentals

## start without pydantic

In [2]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        self.name = name
        self.gender = gender
        self.age = age

person1 = Person(name= "Alex", gender = "M", age= 40)
person1.name, person1.age

('Alex', 40)

In [3]:
person2 = Person(name= 3213+3213, gender = True, age= "minus ten")
person2

## validation of our Person class

In [4]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"name must be of type string, not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age

try:
    person3 = Person(name= 3.1415, gender = "M", age= 40)
except TypeError as err:
    print(err)

name must be of type string, not <class 'float'>


In [5]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"name must be of type string, not {type(name)}")
        self.name = name
        self.gender = gender
        if not isinstance(age, int):
            raise TypeError(f"age must be of type int, not {type(age)}")
        if not 0 <= age < 125:
            raise ValueError(f"age must be between 0 and 124, not {age}")
        self.age = age

    def __repr__(self):
        return f"Person {self.name}, {self.gender}, {self.age}"

try:
    person4 = Person(name= "ragnar", gender = "M", age= -54)
except ValueError as err:
    print(err)
except TypeError as er:
    print(er)



age must be between 0 and 124, not -54


In [6]:
person5 = Person(name= "Bella", gender = "F", age= 4)
person5

Person Bella, F, 4

In [7]:
person5.age = -5
person5.age

# age blir -5 för valideringen (när den testar att age är mellan 0 - 125) är bara inom __init__

-5

In [8]:
class Person:
    def __init__(self, name: str, gender: str, age: int) -> None:
        if not isinstance(name, str):
            raise TypeError(f"name must be of type string, not {type(name)}")
        self.name = name
        self.gender = gender
        self.age = age

    @property # getter
    def age(self):
        return self._age

    @age.setter # setter
    def age(self, age):
        if not isinstance(age, int):
            raise TypeError(f"age must be of type int, not {type(age)}")
        if not 0 <= age < 125:
            raise ValueError(f"age must be between 0 and 124, not {age}")
        self._age = age

    def __repr__(self):
        return f"Person {self.name}, {self.gender}, {self.age}"

person6 = Person(name= "nataly", gender = "F", age= 4)
person6

Person nataly, F, 4

In [9]:
try:
    person6.age = -3
except ValueError as err:
    print(err)

age must be between 0 and 124, not -3


## Validate using pydantic

In [13]:
from pydantic import BaseModel

class Person(BaseModel):
    name: str
    gender: str
    age: int

person7 = Person(name = "christina", gender = "F", age = 29)
person7

Person(name='christina', gender='F', age=29)

In [14]:
person7.age = 36
person7

Person(name='christina', gender='F', age=36)

In [ ]:
person7.age = "thirty" # issue
person7

Person(name='christina', gender='F', age='thirty')

In [16]:
from pydantic import ValidationError
try:
    Person(name = 8474, gender = "F", age = 29)
except ValidationError as err:
    print(err)

1 validation error for Person
name
  Input should be a valid string [type=string_type, input_value=8474, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [ ]:
Person(name = "nina", gender = "F", age = "29") # "29" är coerced till 29

Person(name='nina', gender='F', age=29)

In [21]:
from pydantic import ConfigDict
class Person(BaseModel):
    name: str
    gender: str
    age: int

    model_config = ConfigDict(validate_assignment=True)

person8 = Person(name = "nina", gender = "F", age = "29")
try:
    person8.age = "thirty"
except ValidationError as err:
    print(err)

person8.age = 10
person8

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='thirty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


Person(name='nina', gender='F', age=10)

In [22]:
try:
    person9 = Person(name= 3.1415, gender = 2.7, age= -4)
except ValidationError as err:
    print(err)

2 validation errors for Person
name
  Input should be a valid string [type=string_type, input_value=3.1415, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type
gender
  Input should be a valid string [type=string_type, input_value=2.7, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [24]:
try:
    person9 = Person(name= "sofia", gender = "F", age= -4)
except ValidationError as err:
    print(err)

## adding age validation

In [26]:
from pydantic import Field
from typing import Literal

class Person(BaseModel):
    name: str
    gender: Literal["M", "F"]
    age: int = Field(gt = -1, lt = 125)

    model_config = ConfigDict(validate_assignment=True)

try:
    person9 = Person(name= "sofia", gender = "fem", age= -4)
except ValidationError as err:
    print(err)

2 validation errors for Person
gender
  Input should be 'M' or 'F' [type=literal_error, input_value='fem', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/literal_error
age
  Input should be greater than -1 [type=greater_than, input_value=-4, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than


## Serialization and deserialization

In [ ]:
# serialization, gör ett objekt till json
person8.model_dump()

{'name': 'nina', 'gender': 'F', 'age': 10}

In [29]:
import json
with open("person.json", "w") as file:
    json.dump(person8.model_dump(), file)

In [37]:
with open("person.json", "r") as file:
    json_data = file.read()

print(repr(json_data))
# deserialization
Person.model_validate_json(json_data)


'{"name": "nina", "gender": "F", "age": 10}'


Person(name='nina', gender='F', age=10)